In [26]:
import re
import pandas as pd
import numpy as np
from pathlib import Path

# Functions

In [2]:
#Code snippet: https://programminghistorian.org/en/lessons/generating-an-ordered-data-set-from-an-OCR-text-file

'''
Code ripped from https://www.datacamp.com/community/tutorials/fuzzy-string-python
'''
def lev(seq1, seq2):
    """ levenshtein_ratio_and_distance:
        For all i and j, distance[i,j] will contain the Levenshtein
        distance between the first i characters of seq1 and the
        first j characters of seq2
    """
    # Initialize matrix of zeros
    rows = len(seq1)+1
    cols = len(seq2)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if seq1[row-1] == seq2[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions

    return distance[row][col]

# Code

## Section Tagging GEG

Manual changes:  
- In file GEG_1873_New.txt I manually removed additional spaces from line 632 and line 1626.  
- In file GEG_1874_New.txt I manually removed additional spaces from line 3747 and line 5449 and line 6234.  
- In file GEG_1875_New.txt I manually removed additional spaces from line 957 and line 5255.  
- In file GEG_1875_New.txt I manually added a line break in line 6016 after "Verzeichnis ".
- In file GEG_1876_New.txt I manually removed additional spaces from line 7158.
- In file GEG_1876_New.txt I manually added a line break in line 8200 after "Verzeichnis ".
- In file GEG_1877_New.txt I manually removed additional spaces from line 1790.
- In file GEG_1877_New.txt I manually added a line break in line 5130 after "Verzeichnis ".
- In file GEG_1878_New.txt I manually removed additional symbols bevor and after the content from line 5052.
- In file GEG_1878_New.txt I manually added a line break in line 8490 after "Verzeichnis ".
- In file GEG_1879_New.txt I manually added a line break in line 7390 after "Verzeichnis ".

In [89]:
#I read the file into a list
filename = 'Sources/GEG/1879_GEG_New.txt'
with open(filename, 'r') as fp:
    text = fp.readlines()
    

In [90]:
#I check whether all the sections I want to tag are being detected. 
#If I do notice missing sections I have to check the file manually and adapt the lines if needed.

for num, line in enumerate(text, 1):
    
    yearlymeet_lev_score = lev(line,'Jahresversammlung \n')                            
    section_lev_score = lev(line, '460* Sitzung \n')                                    #I adjust this as best I can to what the ideal outcome would be
    list_lev_score = lev(line, 'Verzeichnis \n')
    
    if yearlymeet_lev_score <= 5:                                                       #I tweak the score until it fits best
        print(f"YOccurence: {line}; Score: {yearlymeet_lev_score}; in Line: {num}")
    elif section_lev_score <= 7:                                                        #I tweak the score until it fits best
        print(f"SOccurence: {line}; Score: {section_lev_score}; in Line: {num}")
    elif list_lev_score <= 4:                                                           #I tweak the score until it fits best
        print(f"LOccurence: {line}; Score: {list_lev_score}; in Line: {num}")
    

YOccurence: Jahresversammlung 
; Score: 0; in Line: 93
SOccurence: 459. Sitzung 
; Score: 3; in Line: 896
SOccurence: 460. Sitzung 
; Score: 1; in Line: 1993
SOccurence: 461. Sitzung 
; Score: 2; in Line: 3746
SOccurence: 462. Sitzung 
; Score: 2; in Line: 4004
SOccurence: 463. Sitzung 
; Score: 2; in Line: 4807
SOccurence: 464i Sitzung 
; Score: 2; in Line: 5280
SOccurence: 465. Sitzung 
; Score: 2; in Line: 5709
SOccurence: 466. Sitzung 
; Score: 2; in Line: 6515
LOccurence: Verzeichnis 
; Score: 0; in Line: 7390
LOccurence: Verzeichnis 
; Score: 0; in Line: 7698
LOccurence: B erzeichniß. 
; Score: 4; in Line: 7911


In [91]:
#I create a new list with the lines that mark sections being tagged.
#I have to adjust the changes made above

new_text = []

for num, line in enumerate(text, 0):
    
    yearlymeet_lev_score = lev(line,'Jahresversammlung \n')                            
    section_lev_score = lev(line, '460* Sitzung \n')                     #I adjust this as best I can to what the ideal outcome would be
    list_lev_score = lev(line, 'Verzeichnis \n')
    
    if yearlymeet_lev_score <= 5:
        ano_line = f"<section type='yearlymeet'> {line}"                 #I open the section tag in the line I detected
        new_text.append(ano_line)
         
    elif section_lev_score <= 7:
        new_text.append("</section> \n")                                 #I close the section tag with an additional line before the new section starts 
        ano_line = f"<section type='monthlymeet'> {line}"
        new_text.append(ano_line)
        
    elif list_lev_score <= 4:
        new_text.append("</section> \n")
        ano_line = f"<section type='list'> {line}"
        new_text.append(ano_line)
        
    else:
        new_text.append(line)
        
new_text.append("</section> \n")                                         #I close the very last section 
        

In [92]:
#I cross check whether everythin works adjusting the numbers so I can see the area surrounding the first monthlymeet
new_text[880:910]

['Woten, die Esten und Liven und,, dürfen wir auch \n',
 'wohl hinzufügen, die Kuren noch eine Einheit bilde-\n',
 'ten, da wohnten sie in unmittelbarer Nachbarschaft \n',
 'der gothisch-nordgermanischen Bevölkerung; w o da-\n',
 'mals beider Wohnsitz war, das zu bestimmen, bietet \n',
 'die Sprachwissenschaft noch kein Mittel. Ob nun \n',
 'aber die eigentliche Geschichte oder die Alterthums-\n',
 'Wissenschaft ihrerseits nicht etwa auch zur Beantwor-\n',
 'tung jener Frage noch etwas beizutragen vermögen, \n',
 'das untersuche ich heute nicht weiter: meine Aufgabe \n',
 'sollte nur sein, vom Boden der Sprachwissenschaft \n',
 'aus den Blick einmal in uralte Völkerverhältnisse \n',
 'und Völkerbeziehungen zu richten, wie sie die Ge-\n',
 'lehrte Estnische Gesellschaft in ganz besonderer Weise \n',
 'zu interessiren wohl geeignet sein dürften. \n',
 '</section> \n',
 "<section type='monthlymeet'> 459. Sitzung \n",
 'der Gelehrteu Estnischen Gesellschaft \n',
 'den 7. (19.) Februar 1879

In [93]:
#I write the new annotated list into a .txt file 

out_filename = 'Output/GEG/1879_GEG_Ano.txt'                            #Look out for the right year!!

filepath = Path(out_filename)                             
filepath.parent.mkdir(parents=True, exist_ok=True)                       #creates the folders if they don't exist yet

with open(out_filename, 'x') as fp:
    fp.write("".join(str(item) for item in new_text))